In [1]:
import os
os.chdir(r'D:\OneDrive - Univerzita Komenskeho v Bratislave\03.PhD\Projects\1.Chapter')
import ee, geemap
#ee.Authenticate()
ee.Initialize()

import sys
sys.path.append('code/helpers/')

from Wrapper import getData

In [2]:
aoiName = 'Carpathians' #Carpathians' #Alps

AOI = (ee.FeatureCollection('projects/ee-simonopravil/assets/LULC/Alps_Carph')
       .filter(ee.Filter.eq('m_range', aoiName))
)

In [3]:
image = getData(aoiName, False)
lc = ee.Image('projects/ee-simonopravil/assets/LULC/EuropeLandCover3').remap([1,2,3,4,5,6,7,8,9,10,11,12], [1,2,2,3,3,3,4,5,6,7,8,9]).rename('class')
lc = lc.clip(AOI)

In [5]:
samples = lc.stratifiedSample(
    numPoints = 0,
    classBand = 'class',
    region = AOI.geometry(),
    scale = 10,
    projection = 'EPSG:3035',
    seed = 3,
    classValues = [1,2,3,4,5,6,7,8],
    classPoints = [70, 200, 640, 70, 220, 70, 70, 70],
    geometries  = True
)

In [6]:
task =  ee.batch.Export.table.toDrive(collection = samples, 
                                      description = f'parametrisation_{aoiName}', 
                                      folder = 'geeExport',
                                      fileFormat = 'SHP')
task.start()

In [15]:
task =  ee.batch.Export.table.toAsset(collection = samples, 
                                      description = 'samples_Alps_prop_1410', 
                                      assetId  = 'projects/ee-simonopravil/assets/LULC/Alps/ValidationSamples_Alps_1410')
task.start()